In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import csv
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# JSON 파일에서 데이터 읽어오기
def read_data_from_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

# 모델 및 토크나이저 초기화
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# GPU를 사용할 수 있다면 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# JSON 파일에서 데이터 읽기
train_data = read_data_from_json('/content/drive/MyDrive/BDP_KoChatGPT/bc_input.json')

# 모델 학습을 위한 설정
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
for example in train_data:
    title = example['title']
    prompt = example['prompt']
    completion = example['completions']
    
    input_text = prompt[:1024]
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    labels = input_ids.clone()
    input_ids = input_ids[:, :1024]
    labels = labels[:, :1024]

    # 모델 학습
    outputs = model(input_ids, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

def predict_completion(prompt):
    encoded_prompt = tokenizer.encode(prompt, truncation=True, max_length=100)

    # 입력 시퀀스가 모델의 최대 길이보다 긴 경우 잘라내기
    if len(encoded_prompt) > 1024:
        encoded_prompt = encoded_prompt[:1024]

    input_ids = torch.tensor(encoded_prompt).unsqueeze(0).to(device)
    output = model.generate(input_ids, max_length=100)
    completion = tokenizer.decode(output[0], skip_special_tokens=True)
    return completion

# 예측 결과를 저장할 csv 파일 열기
with open('/content/drive/MyDrive/BDP_KoChatGPT/completions.csv', 'w', newline='') as f:
    writer = csv.writer(f)

    # 테스트 데이터에 대해 예측하고 csv 파일에 저장
    test_data = read_data_from_json('/content/drive/MyDrive/BDP_KoChatGPT/bc1_input.json')
    for example in test_data:
        title = example['title']
        prompt = example['prompt']
        
        if len(prompt) > 1024:
            prompt_parts = [prompt[i:i+1024] for i in range(0, len(prompt), 1024)]
        else:
            prompt_parts = [prompt]
        
        predicted_completions = []
        for part in prompt_parts:
            predicted_completion = predict_completion(part)
            predicted_completions.append(predicted_completion)
        
        writer.writerow([title] + predicted_completions)

print("Predictions saved to completions.csv")


Token indices sequence length is longer than the specified maximum sequence length for this model (54031 > 1024). Running this sequence through the model will result in indexing errors
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 100, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 100, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The att

Predictions saved to completions.csv


In [ ]:
#학습
import json
import csv
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# JSON 파일에서 데이터 읽어오기
def read_data_from_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

# 모델 및 토크나이저 초기화
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# GPU를 사용할 수 있다면 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# JSON 파일에서 데이터 읽기
train_data = read_data_from_json('/content/drive/MyDrive/BDP_KoChatGPT/bc_input.json')

# 모델 학습을 위한 설정
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
for example in train_data:
    title = example['title']
    prompt = example['prompt']
    completion = example['completions']
    
    input_text =  prompt[:1024] 
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    labels = input_ids.clone()
    input_ids = input_ids[:, :1024]
    labels = labels[:, :1024]

    # 모델 학습
    outputs = model(input_ids, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

Token indices sequence length is longer than the specified maximum sequence length for this model (1858 > 1024). Running this sequence through the model will result in indexing errors
